<a href="https://colab.research.google.com/github/Avonna/ITMO/blob/master/%D0%90%D1%82%D1%82%D0%B5%D1%81%D1%82%D0%B0%D1%86%D0%B8%D1%8F2___%D0%98%D0%A2%D0%9C%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговое задание

Загрузим данные непосредственно из задания (по ссылке).

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
link = 'https://courses.openedu.ru/assets/courseware/v1/431a30c4a3a2b3f265f8cce1a809aa0f/asset-v1:ITMOUniversity+DATANTECH2035+cifru_2035+type@asset+block/pulsar_stars_new.csv'
stars = pd.read_csv(link)

Убедимся, что данные загружены корректно

In [2]:
stars.head()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TG
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


Выберем только интересующие нас данные

In [4]:
stars_train = stars[((stars['TG'] == 0) & (stars['MIP'] >= 99.5234375) & (stars['MIP'] <= 100.109375)) | ((stars['TG'] == 1) & (stars['MIP'] >= 77.4921875) & (stars['MIP'] <= 83.7734375))]
stars_train

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TG
31,99.914062,43.919498,0.475729,0.781486,0.619565,9.440976,20.106639,475.680218,0
277,99.914062,42.729167,0.273129,1.437221,1.813545,13.580166,10.156853,133.133932,0
485,80.640625,56.308821,1.481006,1.952136,27.807692,63.855448,2.179750,3.354892,1
578,80.140625,34.426265,1.697879,6.213364,7.822742,38.341936,5.048004,24.802277,1
595,80.523438,54.076185,1.390950,1.282292,66.311873,77.549955,0.870147,-0.478376,1
...,...,...,...,...,...,...,...,...,...
17026,99.570312,48.399006,0.395253,0.235198,1.287625,14.659487,12.538604,166.286065,0
17140,99.828125,47.239766,0.469564,0.767540,2.816054,16.659173,7.999821,78.263172,0
17492,99.937500,32.513879,0.747187,3.504896,1.537625,11.708532,12.321572,208.082555,0
17772,99.851562,45.877740,0.343986,0.484693,1.272575,12.802890,12.567951,187.122994,0


In [5]:
# Среднее столбца MIP
stars_train.MIP.mean()

90.3051087622549

Выполним линейную нормировку

In [6]:
stars_train_normed = (stars_train - stars_train.min())/(stars_train.max() - stars_train.min())
stars_train_normed

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TG
31,0.991364,0.246152,0.164194,0.113463,0.000000,0.000000,1.000000,1.000000,0.0
277,0.991364,0.223650,0.080687,0.166068,0.007016,0.046570,0.533630,0.282656,0.0
485,0.139206,0.480362,0.578547,0.207376,0.159756,0.612219,0.159724,0.010878,1.0
578,0.117098,0.066691,0.667937,0.549223,0.042325,0.325166,0.294166,0.055792,1.0
595,0.134024,0.438156,0.541428,0.153639,0.386004,0.766296,0.098339,0.002851,1.0
...,...,...,...,...,...,...,...,...,...
17026,0.976166,0.330833,0.131024,0.069639,0.003925,0.058714,0.645268,0.352081,0.0
17140,0.987565,0.308919,0.161653,0.112345,0.012906,0.081212,0.432524,0.167748,0.0
17492,0.992401,0.030539,0.276083,0.331942,0.005394,0.025512,0.635095,0.439609,0.0
17772,0.988601,0.283171,0.109892,0.089654,0.003837,0.037825,0.646644,0.395717,0.0


In [7]:
# Среднее столбца MIP после нормировки
stars_train_normed.MIP.mean()

0.566512580852721

Построим классификатор на основе логистической регрессии

In [8]:
# Предикторы
X = pd.DataFrame(stars_train_normed.drop(['TG'], axis=1))
# Отклики
y = stars_train_normed.TG

In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y)

Выполним предсказание для новой звезды

In [10]:
new_star = [0.121, 0.009, 0.333, 0.335, 0.655, 0.745, 0.736, 0.536]
clf.predict_proba([new_star])
print('Вероятность отнесения к классу пульсар: ', clf.predict_proba([new_star])[0][1])

Вероятность отнесения к классу пульсар:  0.9087268930914426


In [11]:
clf.predict([new_star])

array([1.])

KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier
# Создаем объект класса KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 1, p = 2)
# Обучаем классификатор на тренировочных данных
neigh.fit(X, y)

KNeighborsClassifier(n_neighbors=1)

In [13]:
print('Предсказанный класс: ', neigh.predict([new_star])[0])
print('Расстояние до ближайшей звезды: ', neigh.kneighbors([new_star])[0][0][0])

Предсказанный класс:  1.0
Расстояние до ближайшей звезды:  0.9232341746472038
